### Trabalho de Ciência de Dados


In [1]:
#df.to_csv('PokeData')

In [2]:
# Bibliotecas para análises
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import csv

# Bibliotecas para plotagens
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Exibir versão do Python
import platform
platform.python_version()

'3.9.12'

In [4]:
#Endereço do site
url = 'https://pokemondb.net/pokedex/all'

# Gerando uma lista de user agent (Simulando usos)

user_agent_list = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36',
            'Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/80.0.3987.95 Mobile/15E148 Safari/605.1'
                   
]
# Função parar evitar acessos com o mesmo agent (ada execução ele escolhe um aleatoriamente)
user_agent = random.choice(user_agent_list)

# Cabeçalho
headers = {'User-Agent' : user_agent}


try:
    # Requisição (Sucesso = 200; Erro = 400)
    response = requests.get(url, headers = headers, timeout = 3) 
    response.raise_for_status()
    
    # obtendo toda a informação(content) do site em formato texto/html
    content = response.text
    
except requests.exceptions.HTTPError as http_error:
    # Retorna o erro do servidor
    print ('HTTP Error: ', http_error)
    
except requests.exceptions.ConnectionError as connection_error:
    # Retorna caso esteja sem conexão
    print ('Error Connecting: ', connection_error)
    
except requests.exceptions.Timeout as timeout_error:
    # Retorna caso o servidor não responda no tempo informado anteriormente
    print ('Timeout Error: ', timeout_error)
    
except requests.exceptions.RequestException as generic_error:
    # Caso não ocorra nenhum erro anterior mas continue dando problema, ele irá retornar erro (genérico)
    print ('Generic Error: ', generic_error)

In [5]:
# Usando o BeautifulSoup para transformar o conteúdo de texto/html em um objeto dele
soup = BeautifulSoup(content, 'html.parser')

# Iremos utilizar o método find para achar a tabela com o id: pokedex
table = soup.find(name='table', attrs={'id' : 'pokedex'})

In [1]:
#Esse é todo o conteúdo da tabela
#print(table)

#DE OUTRA FOMRA COM O MESMO RESULTADO
#Achando os filhos da tabela
#for tet in soup.find('table',{'id':'pokedex'}).children:
#    print(tet)

In [7]:
#Salvando toda a tabela numa stinrg
table_str = str(table)

# o read_html ele interpreta as tags da tabela (tr, th e td) e converte isso em um DataFrame
# tr = linha da tabela, th = coluna/cabeçalho e td = valor
PokeData = pd.read_html(table_str)[0]

In [8]:
PokeData

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion Female,Water Ghost,530,120,92,65,100,75,78
1071,903,Sneasler,Poison Fighting,510,80,130,60,40,80,120
1072,904,Overqwil,Dark Poison,510,85,115,95,65,65,85
1073,905,Enamorus Incarnate Forme,Fairy Flying,580,74,115,70,135,80,106


In [9]:
#Ainda é possível fazer a mesma coisa em outra forma, apenas lendo o html da tabela diretamente
#PokeData2 = pd.read_html(content, attrs = {'id' : 'pokedex'})[0]

In [10]:
#PokeData2

### Começar a verificar se o DataFrame está se comportando da maneira esperada para realizar algumas manipulações vistas em aula

In [12]:
#Verificando todas as colunas
for column in PokeData.columns:
    print(column)

#
Name
Type
Total
HP
Attack
Defense
Sp. Atk
Sp. Def
Speed


In [13]:
#Verificando o corpo do DataFrame (n colunas, m linhas)
PokeData.shape

(1075, 10)

In [15]:
#Verificando os tipos de variáveis
PokeData.dtypes

#           int64
Name       object
Type       object
Total       int64
HP          int64
Attack      int64
Defense     int64
Sp. Atk     int64
Sp. Def     int64
Speed       int64
dtype: object

### Iniciando algumas modificações

Temos pokémons que podem possuir 2 tipos, no dataframe original tudo está em uma coluna só, para melhor visualizar podemos separar esses tipos utilizando o pandas, só temos que converter o tipo da coluna para uma string, e utilizamos a função split, caso exista um pokémon que não possua 2 tipos, o tipo 2 será None(nulo)

In [16]:
PokeData[['Tipo 1', 'Tipo 2']] = PokeData['Type'].str.split(' ', expand=True)

In [17]:
PokeData

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Tipo 1,Tipo 2
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80,Grass,Poison
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80,Grass,Poison
4,4,Charmander,Fire,309,39,52,43,60,50,65,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion Female,Water Ghost,530,120,92,65,100,75,78,Water,Ghost
1071,903,Sneasler,Poison Fighting,510,80,130,60,40,80,120,Poison,Fighting
1072,904,Overqwil,Dark Poison,510,85,115,95,65,65,85,Dark,Poison
1073,905,Enamorus Incarnate Forme,Fairy Flying,580,74,115,70,135,80,106,Fairy,Flying


In [19]:
#Excluindo a coluna (Type)
PokeData.drop(columns='Type')

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Tipo 1,Tipo 2
0,1,Bulbasaur,318,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,405,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,525,80,82,83,100,100,80,Grass,Poison
3,3,Venusaur Mega Venusaur,625,80,100,123,122,120,80,Grass,Poison
4,4,Charmander,309,39,52,43,60,50,65,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion Female,530,120,92,65,100,75,78,Water,Ghost
1071,903,Sneasler,510,80,130,60,40,80,120,Poison,Fighting
1072,904,Overqwil,510,85,115,95,65,65,85,Dark,Poison
1073,905,Enamorus Incarnate Forme,580,74,115,70,135,80,106,Fairy,Flying


### Podemos por exemplo fazer uma busca por um tipo específico de Pokémon de acordo com seu tipo

In [29]:
display(PokeData[(PokeData['Tipo 1'] == 'Water') & (PokeData['Tipo 2'] == 'Ghost')])

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Tipo 1,Tipo 2
712,592,Frillish,Water Ghost,335,55,40,50,65,85,40,Water,Ghost
713,593,Jellicent,Water Ghost,480,100,60,70,85,105,60,Water,Ghost
1069,902,Basculegion Male,Water Ghost,530,120,112,65,80,75,78,Water,Ghost
1070,902,Basculegion Female,Water Ghost,530,120,92,65,100,75,78,Water,Ghost
